In [2]:
class SVM_general():
    """ Classify binomial separable and non-separable data through linear and non_linear models.
    
        -- Parameter --
            C: determines the number of points that contribute to creating the boundary. 
               (Default = 0.1)
               The bigger the value of C, the lesser the points that the model will consider.
        
            kernel: name of the kernel that the model will use. Written in a format string 
                    (Default = "linear"). 
        
                    acceptable parameters: 
                        "linear", "poly", "polynomial", "rbf", 
                        "laplacian", "cosine".
        
                    for more information about individual kernels, visit the 
                    sklearn pairwise metrics affinities and kernels user guide.

        --Methods--
            fit(X, y): Learn from the data. Returns self.

            predict(X_test): Predicts new points. Returns X_test labels.

            coef_(): Returns linear model w and b coefficients, or w, x, and b
                     for non_linear models

            For more information about each method, visit specific documentations.
            
        --Example-- 
            ## Calls the classes in SVMLibrary_generalkernel notebook
            >>> %run SVMLibrary_generalkernel.ipynb
            ...
            ## Initializes the object with custom parameters
            >>> model = SVM_general(C = 0.01, kernel = "linear")
            ...
            ## Uses the model to fit the data
            >>> fitted_model = model.fit(X, y)
            ...
            ## Predicts with the given model
            >>> y_prediction = fitted_model.predict(X_test)
            ...
            ## e.g
            >>> print(y_prediction)
            np.array([1, 1, 1, 0, 0, 1, 0])
    """
    
    def __init__(self, C=0.1, kernel = "linear"):
        import numpy as np
        from sklearn.metrics.pairwise import pairwise_kernels
        from cvxopt import solvers, matrix, sparse
        self.C = C
        self.pairwise_kernels = pairwise_kernels
        self.kernel = kernel
        self.matrix = matrix
        self.solvers = solvers
        self.np = np
        
    # learn   
    def fit(self, X, y):
        """ Computes coefficients for the new data prediction.
        
            --Parameters--
                X: nxm matrix that contains all data points
                   components. n is the number of points and
                   m is the number of features of each point.
                   
                y: nx1 matrix that contain the labels for all
                   the points.
            
            --Returns--
                self, containing all the parameters needed to 
                compute new data points: x_k, w, b, and c and a
                which indicate the original label and the label
                used inside the model (e.g [0, 1]->[1, -1])
        """
        def preprocess(X, y):
            # "additive_chi2", "chi2", "sigmoid"
            # label preprocessing
            a = np.unique(y); c = np.array([1, -1])
            y = np.where(y == a[0], c[0], c[1])
            self.y = y
            
            # pre_matrices
            H = self.pairwise_kernels(X, X, metric = self.kernel); Y = np.outer(y, y)
            Q = np.multiply(Y, H) 
            q = -np.ones(y.shape)
            A = np.array(y.reshape(1, -1), dtype = "float64"); b = 0.0
            ydim = y.shape[0]
            G = np.concatenate((np.identity(ydim), -np.identity(ydim)))
            h_ = np.concatenate((self.C*np.ones(ydim), np.zeros(ydim))) 
            h = h_.reshape(-1, 1)
            # return self for later use on predict method
            self.c = c; self.a = a
            return {"H":H, "Q":Q, "q":q, "A":A, "b":b, "G":G, "h":h}
        
        def solver(pre):
            # matrices for the solver
            matrix = self.matrix
            Q = matrix(pre["Q"]); q = matrix(pre["q"])
            A = matrix(pre["A"]); b = matrix(pre["b"])
            G = sparse(matrix(pre["G"])); h = matrix(pre["h"])
            # solver
            solvers = self.solvers
            solvers.options['show_progress']=False
            sol=solvers.qp(P=Q, q=q,G=G,h=h, A=A, b=b)
            return sol
        
        def get_coefficients(sol):
            # alphas threshhold and svs
            alphas = np.array(sol['x']); indx = alphas > 1e-10 
            alpha_sv = alphas[indx]
            y_sv = self.y[indx[:,0]]; x_sv = X[indx[:,0],:]

            # a_k * y_k * K_k
            h_sv = self.pairwise_kernels(x_sv, x_sv, metric = self.kernel)
            w = (alpha_sv*y_sv).reshape(-1, 1)
            # w and b
            w_phi = w.T @ h_sv
            b = np.mean(y_sv - w_phi)
              
            
            # keep on memory for leter use
            self.x_sv = x_sv; self.w = w; self.b = b
            
        # run all previous functions inside fit method
        def run():
            # preprocess the labels and 
            # get the prematrice
            param = preprocess(X, y)
            
            # solve for the alphas
            sol = solver(param)
            
            #compute the coefficients
            get_coefficients(sol)
            
            return self
        
        
        return run()
    
    # predict
    def predict(self, X_test):
        """Predicts new labels for a given set of new 
           independent variables (X_test).
           
           --Parameters--
               X_test: nxm matrix containing all the points that 
                       will be predicted by the model.
                       n is the number of points. m represents the
                       number of features/dimensions of each point.
            
           --Returns--
               a nx1 vector containing the predicted labels for the 
               input variables.
               
               
           --Example-- 
                ## Calls the classes in SVMLibrary_generalkernel notebook
                >>> %run SVMLibrary_generalkernel.ipynb
                ...
                ## Initializes the object with custom parameters
                >>> model = SVM_general(C = 0.01, kernel = "linear")
                ...
                ## Uses the model to fit the data
                >>> fitted_model = model.fit(X, y)
                ...
                ## Predicts with the given model
                >>> y_prediction = fitted_model.predict(X_test)
                ...
                ## e.g
                >>> print(y_prediction)
                np.array([1, 1, 1, 0, 0, 1, 0])
                
        """
        np = self.np
        # rename label variables
        c = self.c; a = self.a
        # rename coefficients
        b = self.b; x_sv = self.x_sv
        
        # create new kernel
        H = self.pairwise_kernels(x_sv, X_test, metric = self.kernel)
        # multiply w and kernel
        w_phi = self.w.T @ H
        
        # predict new data
        predict1 = np.sign(w_phi + b)
        # rename to original labels
        predict2 = np.where(predict1 == c[0], a[0], a[1])
        return predict2
    
    # coefficient
    def coef_(self):
        np = self.np
        if self.kernel == "linear":
            w = self.w; x_sv = self.x_sv
            w = np.sum(np.multiply(w, x_sv), axis = 0)
            return w, self.b
        else: 
            return self.w, self.x_sv,  self.b

In [5]:
class plott:
    @staticmethod
    def line(X, w, b):
        xmin, xmax = min(X[:,0])-1, max(X[:,0]+1)
        X_ = np.arange(xmin, xmax, 0.1)
    
        plt.plot(X_,(-w[0]*X_-b)/w[1])
    
    @staticmethod
    def scatter(X, y, m = "o"):
        plt.scatter(X[:,0], X[:,1], c = y, marker = m)

In [20]:
class utilss:
    @staticmethod
    def contour_plot(X, y, kernel="linear", C = 0.1):
        h = .02
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h),np.arange(y_min, y_max, h))
        
        Z = SVM_general(C = C, kernel = kernel).fit(X, y).predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)

        plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
        
    @staticmethod
    def X_random(X, n_predict):
        max_min = np.array([[X[:,k].min(), X[:,k].max()] for k in range(X.shape[1])])
        X_test0 = np.random.choice(np.arange(round(max_min[0][0]), round(max_min[0][1]), 0.1), n_predict)
        max_min = np.delete(np.array(max_min),0,  axis = 0)
        for i in max_min:
            X_test1 = np.random.choice(np.arange(round(i[0]), round(i[1]), 0.1), n_predict)
            X_test0 = np.c_[X_test0, X_test1]
            X_test0.astype("float64")
        return X_test0